In [1]:
#загружаем необходимые библиотеки
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.stats import norm
from scipy.stats import ttest_1samp
from scipy.stats import levene
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

In [2]:
#открываем файлы
df04oes = pd.read_csv('04oes.csv', encoding = 'utf-8')
df06m06 = pd.read_csv('06m06.csv', encoding = 'utf-8')
df10p39 = pd.read_csv('10p39.csv')
df04shk = pd.read_csv('04shk.csv')
df05m35 = pd.read_csv('05m35.csv', encoding = 'utf-8')
df06p13 = pd.read_csv('06p13.csv', encoding = 'utf-8')

In [3]:
df1 = df06p13
df1.head() #что из себя представляет таблица 06p13?

,Unnamed: 0,Q7,Q8,Q48_1,Q48_2,Q48_3,Q48_4,Q48_5,Q48_6,Q48_7,Q48_8,Q48_9,Q48_10,Q9,Q11,Q15,Q64,Q13
0,1,женский,50,0,"""Золотая семечка""",0,0,0,0,0,0,0,0,"высшее, незаконченное высшее","служащий, технический исполнитель",безусловно не смогу,большой город,3000.0
1,2,женский,77,0,0,0,0,0,0,0,0,0,0,начальное или ниже,NaN,безусловно не смогу,большой город,2000.0
2,3,мужской,48,"""Злато""","""Золотая семечка""",0,0,0,0,0,0,0,0,среднее специальное,рабочий,безусловно не смогу,большой город,1700.0
3,4,женский,52,0,"""Золотая семечка""",0,0,"""Милора""",0,0,0,0,0,среднее специальное,NaN,безусловно не смогу,большой город,4000.0
4,5,мужской,46,0,0,0,0,0,0,0,0,0,0,среднее специальное,"служащий, технический исполнитель",скорее не смогу,большой город,2000.0


In [4]:
df1['Q48_6'].unique() #интересующий нас вопрос анкеты представлен строковой переменной(!)

array(['0', '"Моя семья"'], dtype=object)

In [5]:
df1['Q7'].unique() #смотрим, какие значения принимает переменная "Пол" (Q7)

array(['женский', 'мужской'], dtype=object)

In [6]:
df1['Q8'].sort_values().unique() #смотрим, какие значения принимает переменная "Возраст" (Q8)

array([18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 90], dtype=int64)

In [7]:
#создаём новую бинарную переменную, которая принимает значение "1" для нашей целевой группы (мужчины в возрасте от 18 до 35) и "0" - во всех остальных случаях
df1.loc[(df1['Q7'] == 'мужской') & (df1['Q8'] <= 35), 'New'] = 1
df1['New'].fillna(0, inplace = True)

In [8]:
#pd.crosstab([df1['Q7'],df1['Q8']], df1['New'], margins = True) так можно сделать одну таблицу сопряжённости для 3 переменных
pd.crosstab(df1['Q7'], df1['New'], margins = True) #таблица сопряжённости "Пол" vs "New"

New,0.0,1.0,All
Q7,,,
женский,1091,0,1091
мужской,574,351,925
All,1665,351,2016


In [9]:
pd.crosstab(df1['Q8'], df1['New'], margins = True) #таблица сопряжённости "Возраст" vs "New"

New,0.0,1.0,All
Q8,,,
18,18,36,54
19,23,17,40
20,24,21,45
21,15,18,33
22,21,12,33
23,18,21,39
24,23,21,44
25,15,21,36
26,16,18,34


In [10]:
#выставляем фильтр по нашей новой переменной, чтобы рассматривать информированность о продукте только среди целевой группы:
newdf1 = df1.loc[df1['New'] == 1]

In [11]:
#заменим значения переменной Q48_6 на цифры, чтобы было можно посчитать среднее:
newdf1['Q48_6'] = newdf1['Q48_6'].replace('"Моя семья"', 1)
newdf1['Q48_6'] = newdf1['Q48_6'].replace('0', 0)
newdf1['Q48_6'].unique()#проверка

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


array([1, 0], dtype=int64)

2.1. информированность о марке подсолнечного масла "Моя семья" среди мужчин 18 – 35 лет превышает 38% (?) 06p13 "z-test" & t-test

In [12]:
#можно не создавать новую переменную, а сразу отобрать наблюдения для целевой группы:
df1_1 = df1[df1['Q7'] == 'мужской'][df1['Q8'] < 36]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [13]:
df1_1['Q48_6'] = df1_1['Q48_6'].replace('"Моя семья"', 1)
df1_1['Q48_6'] = df1_1['Q48_6'].replace('0', 0)

In [56]:
#Z-test pryamaya shema:
p_zad = 0.38
n = len(df1_1[df1_1['Q48_6'] == 1])
N = len(df1_1['Q48_6'])
alpha = 0.05

#(alpha - (1 - norm.cdf(((n/N) - p_zad)/(((p_zad*(1-p_zad))/N)**.5)))) < 0
p_vyb = n/N
stdev_vyb = ((p_zad*(1-p_zad))/N)**.5
Z_star = (p_vyb - p_zad)/stdev_vyb
VerH0 = 1 - norm.cdf(Z_star)
(alpha - VerH0) < 0

False

In [60]:
#Z-test obratnaya shema:
p_zad = 0.38
n = len(df1_1[df1_1['Q48_6'] == 1])
N = len(df1_1['Q48_6'])
alpha = 0.05

#((n/N) - (norm.ppf(1-alpha) * ((p_zad*(1-p_zad))/N)**.5 + p_zad)) < 0
Z_alpha = norm.ppf(1-alpha)
stdev_vyb = ((p_zad*(1-p_zad))/N)**.5
p_alpha = Z_alpha * stdev_vyb + p_zad
p_vyb = n/N
(p_vyb - p_alpha) < 0

False

# Мы отклоняем нулевую гипотезу и принимаем альтернативную: осведомлённость выше 38%

In [75]:
#Одновыборочный тест Стьюдента. проверяем гипотезу Н0 о равенстве долей выборки и ГС (двусторонний тест):
ttest_1samp(df1_1['Q48_6'], 0.38)

Ttest_1sampResult(statistic=1.6844038786290607, pvalue=0.09299512142993895)

In [76]:
(ttest_1samp(df1_1['Q48_6'], 0.38)[1]/2) > 0.05

False

# Мы отклоняем нулевую гипотезу и принимаем альтернативную: осведомлённость выше 38%

2.2. организация работы раздевалок нравилась более чем 61% бывших клиентов (?) 04oes z-test & t-test
исключить в 12-м вопросе "не посещал" и "нет ответа"

In [77]:
df04oes = pd.read_csv('04oes.csv', encoding = 'utf-8')
df2 = df04oes[(df04oes['m12'] != 'не посещал(-а)') & (df04oes['m12'] != 'Нет ответа')]
for i in df2['m12'].unique():
    if (i == 'безусловно нравилось' or i == 'скорее нравилось'):
        df2['m12'] = df2['m12'].replace(i, 1)
    else:
        df2['m12'] = df2['m12'].replace(i, 0)
v04oes = df2['m12'].values

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [78]:
ttest_1samp(v04oes, 0.61)

Ttest_1sampResult(statistic=1.5451911578460809, pvalue=0.12379851846576297)

In [79]:
(ttest_1samp(v04oes, 0.61)[1]/2) > 0.05

True

# на данном уровне значимости нам не удалось отклонить нулевую гипотезу. организация работы раздевалок нравилась не более чем 61% бывших клиентов 

3.1. №8 и №36. различия в доходах, 2 3 балла и 4 5 по истории. Independent samples, means. Levene's test for equality of variances --> t-test 4 equality of means 4shk

In [80]:
df3 = pd.read_csv('04shk.csv')
for i in df3['q8'].unique():
    if (i == 4 or i == 5):
        df3['q8'] = df3['q8'].replace(i, 1)
    elif (i == 2 or i == 3):
        df3['q8'] = df3['q8'].replace(i, 0)
otl = df3[df3['q8'] == 1]['q36'].values
dvo = df3[df3['q8'] == 0]['q36'].values
otl = otl[~np.isnan(otl)]
dvo = dvo[~np.isnan(dvo)]

## SPSS использует только среднее значение для центра в тесте Левиня (а возможны также медиана и 'trimmed')!!!
median - skewed (non-normal) dist.; mean - symmetric, moderate-tailed; trimmed - heavy-tailed

In [81]:
ttest_ind(otl, dvo, equal_var = (levene(otl, dvo, center = 'mean')[1] > 0.05))

Ttest_indResult(statistic=2.3204182136835705, pvalue=0.021506181298543467)

In [82]:
(ttest_ind(otl, dvo, equal_var = (levene(otl, dvo, center = 'mean')[1] > 0.05))[1])/2 > 0.05

False

# Мы отклоняем нулевую гипотезу и принимаем альтернативную: доходы значимо различаются

3.2. №30, 33, 34. траты на обучение в 2004 по сравн. с 2003. "живём неплохо" и ниже в 30-м (paired samples t-test) 4shk

In [83]:
df4 = pd.read_csv('04shk.csv')
df4['q30'].unique()

array(['Живем обеспеченно. Удается не только хорошо питаться, модно ',
       'Живем неплохо. Нормально питаемся, покупаем кое-что из модно',
       'Удается сводить концы с концами, нормально питаемся. Приобре',
       'Часто не хватает средств на самое необходимое, даже на норма',
       'Ни в чем себе не отказываем: ни в еде, ни в одежде, ни в при',
       '0'], dtype=object)

In [84]:
data = df4[(df4['q30'] != 'Ни в чем себе не отказываем: ни в еде, ни в одежде, ни в при')
           & (df4['q30'] != '0')
           & (df4['q30'] != 'Живем обеспеченно. Удается не только хорошо питаться, модно ')]
df5 = data[['q30', 'q33', 'q34']]
df5 = df5.dropna()
prosh = df5['q33'].values
etot = df5['q34'].values

In [85]:
ttest_rel(prosh, etot)

Ttest_relResult(statistic=-6.3757217182337484, pvalue=8.05082377916037e-10)

In [86]:
ttest_rel(prosh, etot)[1] > 0.05

False

# Мы отклоняем нулевую гипотезу и принимаем альтернативную: затраты значимо различаются

In [87]:
# на сколько?
delta = np.mean(etot)-np.mean(prosh)
delta_prcnt = (np.mean(etot)/np.mean(prosh) - 1)*100
correlation = np.corrcoef(prosh, etot)
print(round(delta, 2), round(delta_prcnt, 2), round(correlation[0,1],2))

700.65 48.58 0.78


3.3. №30, 33, 34. траты на обучение в 2004 по сравн. с 2003. выше "живём неплохо" в 30-м (paired samples t-test) 4shk

In [88]:
df4 = pd.read_csv('04shk.csv')
data = df4[(df4['q30'] != 'Часто не хватает средств на самое необходимое, даже на норма')
           & (df4['q30'] != '0')
           & (df4['q30'] != 'Живем неплохо. Нормально питаемся, покупаем кое-что из модно')
           & (df4['q30'] != 'Удается сводить концы с концами, нормально питаемся. Приобре')]
df5 = data[['q30', 'q33', 'q34']]
df5 = df5.dropna()
prosh = df5['q33'].values
etot = df5['q34'].values
ttest_rel(prosh, etot)

Ttest_relResult(statistic=-5.927485434906174, pvalue=1.511452648851762e-08)

In [89]:
ttest_rel(prosh, etot)[1] > 0.05

False

# Мы отклоняем нулевую гипотезу и принимаем альтернативную: затраты значимо различаются

In [91]:
# на сколько?
delta = np.mean(etot)-np.mean(prosh)
delta_prcnt = (np.mean(etot)/np.mean(prosh) - 1)*100
correlation = np.corrcoef(prosh, etot)
print(round(delta, 2), round(delta_prcnt, 2), round(correlation[0,1],2))

1130.03 59.27 0.71
